In [1]:
%%html
<!-- definir quelques styles custom pour l'ensemble du notebook -->
<style>    
    @import url("css/custom_styles.css")
</style>

<center>
    <h1>
    Transformation Et Manipulation Des Données<br>
    </h1>
    MovieLens - Système de recommendations de films par regroupement<br>
    <br>
    <b>Jean-Francois Gagnon</b><br>
    <br>
    420-A56<br>
    <br>
</center>

In [ ]:
#
# imports utilitaires
#

%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from functools import reduce
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import TruncatedSVD

from tqdm.notebook import tqdm

#
# imports faisant partie de nos propres modules
#

import helpers as hlp
import helpers.Clustering as clstr

from helpers.jupyter import display_html

# Exploration

**ATTENTION** il faut exécuter Pretraitement.ipynb avant celui-ci

In [ ]:
#
# parametres configurant nos traitemens
#
configs = hlp.get_configs("config_overrides.json")

In [ ]:
# imdbId doit etre garde en string (leading 0)
movies_df = pd.read_csv("dataset/movies_pretraitement.csv", dtype={"imdbId": "string"})

print("Movies", movies_df.shape)
print("Head")
display(movies_df.head().round(2))
clstr.show_na(movies_df)
clstr.show_types(movies_df)

<font class="answer">

* 9724 individus, 7 variables (movieId, imdbId et *title* sont en fait des identifiants).
* aucune valeur manquante  
* *genres* est de type qualitatif, les autres variables sont quantitatives. Cependant, en regardant les descriptions de MovieLens, on se rend compte que tout est en fait qualitatif (rating est un système d'étoiles avec une granularité de $\frac{1}{2}$).

In [ ]:
quant_cols = movies_df.columns[4:]
quant_df = movies_df[quant_cols]

qual_cols = ["genres"]
qual_df = movies_df[qual_cols]

## Distributions - variables quantitatives

In [ ]:
print("Stats générales - variables quantitatives")
display(quant_df.describe().round(2))

print("Distributions - variables quantitatives")
clstr.show_distributions(quant_df, num_cols=2, figsize=(10, 7))

<font class="answer">
    
### *year*
Les films sont relativement récents: peak autour de l'an 2000. La base de données couvre un large spectre et la distribution est allongée: queue à gauche.
    
### *rating_count*
Les films sont majoritairement votés par peu de personnes. Distribution très alongée avec un très fort peak dans les faibles valeurs. 75% de la poputlation a moins de 9 votes, 25% n'en a qu'un seul. J'interprète cette métrique comme une mesure de la popularité.
    
### *rating_mode*, *rating_mean* et *rating_median* 
Très similaire: la majorité des votes se trouvent entre 3 et 4. Distrubution très près d'une gaussienne.

### *rating_etendue*, *rating_std* et *rating_iqr*

**Ces métriques ont étés enlevés**
    
Ces métriques mesurent la dispersion des votes des utilisateurs pour un film. Je l'interprète comme une mesure de "concensus". 0 marquent les films où il n'y a qu'un seul vote. *rating_etendue* montrent bien que comparer les valeurs des votes n'apporte que peu d'information (toute l'étendue est représentée). *rating_std* et *rating_iqr* semblent plus montrer une tendance. Les votes semblent comparable dans un interval d'environ [0.5, 1.0].

In [ ]:
clstr.show_correlation(quant_df)

# pour simplifier visualisation du pair plot
quant_sampled_df = quant_df.sample(frac=0.2)
sns.pairplot(quant_sampled_df, corner=True);

Ici on devrait trancher: garde year, rating_mean

## Distributions - variables qualitatives

In [ ]:
genres_count = {}
for genres_str in movies_df.genres:
    genre_array = genres_str.split(configs.dataset.genre_splitter)
    for genre in genre_array:
        if genre in genres_count:
            genres_count[genre] += 1
        else:
            genres_count[genre] = 1

genres_df = pd.Series(data=genres_count.values(),
                      index=genres_count.keys(),
                      name="count")
genres_df.sort_values(ascending=False, inplace=True)
print(genres_df)

plt.bar(genres_df.index, genres_df.values)
plt.xticks(rotation=90)
plt.show()

<font class="answer">
    
Forte proportion dans Drame et Comedy

## Valeurs aberrantes

Si on se positione dans un contexte de sugestion, on peu assumer des films récent et populaire donc rating_count, year et rating_mean sont des bons candidats pour filtrer rating_min et rating_max sont trop confus pour tirer des conclusions

In [ ]:
coords = movies_df.rating_count.to_numpy().reshape(-1, 1)
clstr.kmeans_analysis(coords)

In [ ]:
kmeans = clstr.kmeans_init(coords, 2)
kmeans.cluster_centers_

In [ ]:
# ll, _ = clstr.transform_boxcox(movies_df, columns=["rating_count"])

ddd = movies_df.rating_count.to_frame().copy()
ddd["cluster"] = kmeans.labels_

sns.histplot(x="rating_count", data=ddd, hue="cluster")
plt.show()